In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from urllib.request import urlopen, Request
from selenium.webdriver.support import expected_conditions as EC

In [3]:
driver = webdriver.Chrome()
url = 'https://th.investing.com/earnings-calendar/'
driver.get(url)

# Try to close popup if it appears
try:
    popup_close = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".popupCloseIcon.largeBannerCloser"))
    )
    popup_close.click()
    print("Found and clicked the 'Close' button!")
except Exception as e:
    print("Popup close button not found or not clickable:", e)

# Click the Filters button
try:
    filters_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "filterStateAnchor"))
    )
    filters_button.click()
    print("Found and clicked the 'Filters' button!")
except Exception as e:
    print("Filters button not found or not clickable:", e)

# Click the Clear button
try:
    erase_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.LINK_TEXT, "ล้างข้อมูล"))
    )
    erase_button.click()
    print("Found and clicked the 'Clear' button!")
except Exception as e:
    print("Clear button not found or not clickable:", e)

# Click the United States button
try:
    country_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "country5"))
    )
    country_button.click()
    print("Found and clicked the 'United States' button!")
except Exception as e:
    print("United States button not found or not clickable:", e)

# Click the 2 Star button
try:
    star2_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "importance2"))
    )
    star2_button.click()
    print("Found and clicked the '2 Star' button!")
except Exception as e:
    print("2 Star button not found or not clickable:", e)

# Click the 3 Star button
try:
    star3_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "importance3"))
    )
    star3_button.click()
    print("Found and clicked the '3 Star' button!")
except Exception as e:
    print("3 Star button not found or not clickable:", e)

# Click the Apply button
try:
    apply_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "ecSubmitButton"))
    )
    apply_button.click()
    print("Found and clicked the 'Apply' button!")
except Exception as e:
    print("Apply button not found or not clickable:", e)


time.sleep(5)   
response = driver.page_source
soup = BeautifulSoup(response, 'html.parser')
table = soup.find_all("tbody")
driver.quit()

Popup close button not found or not clickable: Message: 

Found and clicked the 'Filters' button!
Clear button not found or not clickable: Message: element click intercepted: Element <a href="javascript:void(0);" onclick="clearAll('country[]');">...</a> is not clickable at point (41, 791). Other element would receive the click: <div class="floatingAlertWrapper alertWrapper js-notification-item articles shown" data-item="0">...</div>
  (Session info: chrome=137.0.7151.104)
Stacktrace:
	GetHandleVerifier [0x0x7ff7230ffe75+79173]
	GetHandleVerifier [0x0x7ff7230ffed0+79264]
	(No symbol) [0x0x7ff722eb9e5a]
	(No symbol) [0x0x7ff722f18599]
	(No symbol) [0x0x7ff722f15f32]
	(No symbol) [0x0x7ff722f12f71]
	(No symbol) [0x0x7ff722f11e61]
	(No symbol) [0x0x7ff722f03464]
	(No symbol) [0x0x7ff722f3896a]
	(No symbol) [0x0x7ff722f02d16]
	(No symbol) [0x0x7ff722f38b80]
	(No symbol) [0x0x7ff722f6100d]
	(No symbol) [0x0x7ff722f38743]
	(No symbol) [0x0x7ff722f014c1]
	(No symbol) [0x0x7ff722f02253]
	GetHan

In [4]:
def earnings_calendar(table_bodies):
    data = []
    current_date = None

    for tbody in table_bodies:
        for tr in tbody.find_all("tr"):
            date_td = tr.find(class_="theDay")
            if date_td:
                current_date = date_td.text.strip()
            else:
                company_span = tr.find(class_="earnCalCompanyName middle")
                ticker_a = tr.find(class_="bold middle")
                all_left_strong_tds = tr.find_all(class_="leftStrong")
                market_cap = tr.find(class_="right")
                if tr.find(class_="marketOpen genToolTip oneliner reverseToolTip"):
                    outtime_status = "Before Market"
                elif tr.find(class_="marketClosed genToolTip oneliner reverseToolTip"):
                    outtime_status = "After Market"
                else:
                    outtime_status = "N/A"
                estimate_eps = 'N/A'
                estimate_revenue = 'N/A'
                
                if company_span and ticker_a:
                    company = company_span.text.strip() if company_span else None
                    ticker = ticker_a.text.strip() if ticker_a else None
                    estimate_eps = all_left_strong_tds[0].text.replace('/','').strip() if estimate_eps else None
                    estimate_revenue = all_left_strong_tds[1].text.replace('/','').strip() if estimate_revenue else None
                    market_cap = market_cap.text.strip() if market_cap else None
                    
                    date=datetime.strptime(current_date, '%A, %B %d, %Y')
                    
                    data.append({
                        'date': date,
                        'company': company,
                        'ticker': ticker,
                        'estimate_eps': estimate_eps,
                        'estimate_revenue': estimate_revenue,
                        'market_cap': market_cap,
                        'outtime': outtime_status
                    })
                else:
                    continue
    return data

In [5]:
earnings_calendar_data = earnings_calendar(table)
df = pd.DataFrame(earnings_calendar_data)

In [6]:
df.head(20)

,date,company,ticker,estimate_eps,estimate_revenue,market_cap,outtime
0,2025-06-16,Assura,AGRP,--,--,1.63B,N/A
1,2025-06-16,NextEnergy Solar,NESF,--,--,403.79M,Before Market
2,2025-06-17,Wijaya Karya,WIKA,--,--,8.13T,N/A
3,2025-06-18,Ao World,AO,--,--,550.39M,N/A
4,2025-06-19,XPS Pensions,XPS,--,--,796.9M,Before Market
5,2025-06-19,Urban Logistics,SHED,--,--,715.31M,N/A
6,2025-06-19,Syncona,SYNCS,--,--,538.3M,Before Market
7,2025-06-20,Berkeley,BKGH,--,--,4.24B,N/A
8,2025-06-23,Sdcl Energy Efficiency,SEIT,--,--,532.94M,N/A


In [7]:
now = datetime.now()
date_time = now.strftime("%Y-%m-%d %H-%M-%S").strip().replace(' ', '_')
df.to_csv(f'Data/Earnings_Calendar/{date_time}.csv', index=False)